# Энергетические модели как концепция

Цели:

* Получить более широкий взгляд на концепцию energy-based моделей
* Рассмотреть известные архитектуры моделей с этой точки зрения

Содержание:

* Преимущества EBM
* Связь EBM с физикой
* Заметки по обучению EBM
* Архитектуры моделей
  * Binary classifier
  * AutoEncoder
  * Denoising AutoEncoder
  * Variational AutoEncoder
  * Normalizing flows
  * Diffusion models

Ссылки:

* [Energy-based models for sparse overcomplete representations. Teh, Welling, Osindero, Hinton (2003). ](https://www.jmlr.org/papers/v4/teh03a.html)
* [A Tutorial on Energy-Based Learning. LeCun, Chopra, Hadsell, Ranzato, Huang (2006). ](https://cs.nyu.edu/~sumit/publications/assets/ebmtutorial.pdf)
* [Energy-based generative adversarial network. Zhao, Mathieu, LeCun (2016).](https://arxiv.org/pdf/1609.03126.pdf)
* [Слайды Alfredo Canziani с практического занятия на курсе DLSP21 NYU](https://drive.google.com/file/d/1aa1Hzq5KRekq32mlW4_pgIXMec18WgOg/edit)
* [NYU DLSP21 страничка курса](https://atcold.github.io/NYU-DLSP21/)
* [NYU DLFL22 страничка курса](https://atcold.github.io/NYU-DLFL22/)
* [NYU DLSP21 YT playlist](https://www.youtube.com/watch?list=PLLHTzKZzVU9e6xUfG10TkTWApKSZCzuBI)
* [NYU DLFL22 YT playlist](https://www.youtube.com/watch?list=PLLHTzKZzVU9d_3TcHbyiAjl5qCbpJR-o0)
* https://lilianweng.github.io/posts/2018-10-13-flow-models/
* https://lilianweng.github.io/posts/2021-07-11-diffusion-models/






## Преимущества EBM

Главная цель построения классических моделей машинного обучения - закодировать зависимости между переменными. Модель, которая выучила эти взаимосвязи, может быть использована для ответов на вопросы о значениях неизвестных переменных (Y) при данных известных (X).

EBM ассоциируют скалярную величину энергии, как меру совместимости для каждого набора входных данных (X, Y). Инференс состоит в определении наблюдаемых величин и нахождении значений оставшихся величин так, чтобы энергия была минимальна.

Обучение состоит в поиске энергетической функции, которая ассоциирует низкую энергию для корректных значений оставшихся переменных и высокую для некорректных.

Функционал потерь, минимизируемый во время обучения, используется для определения качества доступных энергетических функций.

E(Y, X) - энергетическая функция, отражающая степень совместимости между X и Y, где хорошо совместимым парам соот. низкие значения энергии.

Следует провести различие между энергетической функцией, которая минимизируется в процессе вывода, и функционалом потерь (введенным в разделе 2), который минимизируется в процессе обучения.

На вход подаётся X и модель производит ответ Y, наиболее совместимый с наблюдаемым X. Более точно, модель должна произвести значение $Y^∗$, выбранное из множества $\mathcal{Y}$, для которого $E(Y, X)$ минимально:
$$Y^* = \arg \min_{Y \in \mathcal{Y}} E(Y, X).$$

Когда размер множества $\mathcal{Y}$ невелик, мы можем просто вычислить $E(Y, X)$ для всех возможных значений $Y ∈ \mathcal{Y}$ и выбрать наименьшее.
Но иногда $\mathcal{Y}$ слишком велико для полного перебора, например повышение качества изображения, перевод и пр..

В таких ситуациях специальная стратегия, именуемая процедурой инференса должна быть использована, для поиска Y минимизирующего энергию. Выбор такой процедуры зависит от данных, например если Y непрерывная величина и энергия гладкая, это может быть алгоритм оптимизации на основе градиента. Или, если энергия представляется в виде графа, то инференс будет представлять собой поиск пути по взвешенному ациклическому графу, где энергия будет стоимость этого пути. Например в задаче распознавания рукописного текста.

Примеры задач, решаемых в парадигме энергетических моделей:

* классификация - насколько Y совместим c X
* ранжирование - кто больше совместим Y1 или Y2
* детекция - обнаружение лиц на фото и отбор областей по порогу
* принятие решений (управление роботом) - хорошие направления продолжения движения имеют наименьшую энергию


## Связь EBM с физикой

Нормируют чаще всего с помощью распределения Больцмана (Гиббса):
$$P(Y | X) = \frac{e^{-\beta E(Y, X)}}{\int_{y \in Y} e^{-\beta E(y, X)} \,dy},
$$

где $β$ - произвольная положительная константа, а знаменатель называется функцией разбиения (partition function) или интеграл состояний или каноническая статистическая сумма, по аналогии с подобными концепциями в статистической физике. Формула очень похожа на softmax.

Распределение Гиббса в физике - это распределение вероятностей, которое даёт вероятность того, что система будет находиться в определённом состоянии как функция энергии этого состояния и температуры системы. Состояния с низкой энергией имеют более высокую вероятность быть занятыми.

Из физики известно, что в системе с фиксированными _температурой_ и _объёмом_ положение устойчивого равновесия соответствует точке минимума свободной энергии.


## Заметки по обучению EBM

Обучение EBM заключается в поиске энергетической функции, которая для любого значения X дает наилучшее Y. Поиск наилучшей энергетической функции выполняется в пределах семейства энергетических функций $\mathcal{E}$, параметризуемых параметром $W$:
$$\mathcal{E} = \{E(W, Y, X) : W \in \mathcal{W}\}$$

Задача обучения - поиск $W^∗$ минимизирующих функционал $\mathcal{L}$ на обучающей выборке $\mathcal{S} = \{(X_i, Y_i) : i = 1, \ldots, P\}$
$$W^∗=\underset{W \in \mathcal{W}}{\min}\mathcal{​L}(W,\mathcal{S})$$
Часто функционал потерь определяется как
$$\mathcal{L}(E, \mathcal{S}) = \frac{1}{P} \sum_{i=1}^{P} L(Y_i, E(W, \mathcal{Y}, X_i)) + R(W)
$$

Это просто среднее значение функционалов по образцам из обучающей выборки, зависящее от желаемого ответа $Y_i$ и энергии, получаемой при фиксированном X и изменении Y. Так для каждого образца мы оцениваем срез поверхности энергии. $R(W)$ - регуляризация, которая может быть использована, чтобы встроить априорные знания о том, какие энергетические функции предпочитаемы в случае отсутствия обучающих данных.


<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/energy_surface.png" width="600">

Энергетическая функция (F) даёт низкие значения для совместимых пар и высокие для несовместимых, что похоже на физику.
Inference - найти такие y, что F(x, y) принимает низкие значения и таких решений может быть несколько. Всё, что от неё нужно - низкая энергия, для того, что мы хотим получать и высокая для тех, что не хотим.

Например, FFNN - явная функция по вычислению y по x, EBM - неявная функция, которая выучивает зависимости между y и x.

Если одному значению x соответствует множество y, то обычной сети будет плохо, а энергетической функции - нормально).

Поиск y - задача оптимизации и может например выполняться градиентным спуском (не стохастическим).

Пример для дискретного случая очень напоминает обучение языковых моделей.

<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/ebm_discrete_case_example.png" width="600">


Суть обучения - **Push down the energy of data points, make sure the energy is higher elsewhere.**

Как этого добиться, есть два способа:

* контрастивные методы - понижаем энергию известных пар (x, y), повышаем энергию остальных пар, простые методы, но менее эффективны
* архитектурные методы / регуляризация - строим F(x, y) так, чтобы объём областей с низкой энергией был ограничен или минимизирован с помощью регуляризации


Слайд с лекции ЛеКуна:

* Contrastive: (they all are different ways to pick which points to push up)
	* C1: push down of the energy of data points, push up everywhere else: Max likelihood (needs tractable partition function or variational approximation)
	* C2: push down of the energy of data points, push up on chosen locations: max likelihood with MC/MMC/HMC, Contrastive divergence, Metric learning/Siamese nets, Ratio Matching, Noise Contrastive Estimation, Min Probability Flow, adversarial generator/GANs
	* C3: train a function that maps points off the data manifold to points on the data manifold: denoising auto-encoder, masked auto-encoder (e.g. BERT)
* Regularized/Architectural: (Different ways to limit the information capacity of the latent representation)
	* A1: build the machine so that the volume of low energy space is bounded: PCA, K-means, Gaussian Mixture Model, Square ICA, normalizing flows...
	* A2: use a regularization term that measures the volume of space that has low energy: Sparse coding, sparse auto-encoder, LISTA, Variational Auto-Encoders, discretization/VQ/VQVAE.
	* A3: F(x,y) = C(y, G(x,y)), make G(x,y) as "constant" as possible with respect to y: Contracting auto-encoder, saturating auto-encoder
	* A4: minimize the gradient and maximize the curvature around data points: score matching

Примеры моделей

Contrastive Joint Embedding - контрастивный метод. Нужно очень много данных (hard negative mining).

<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/contrastive_joint_embedding.png" width="300">

Barlow twins - похожий, но не контрастивный.

<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/barlow_twins.png" width="600">

## Архитектуры моделей

### Binary classifier

![AE](https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/classifier_arch.png)

### AutoEncoder

![AE](https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/ae_arch.png)

### Denoising AutoEncoder

![AE](https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/dae_arch.png)

### Variational AutoEncoder

![AE](https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/vae_arch.png)

### Normalizing flows

![AE](https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/nf_arch.png)

### Diffusion models

![AE](https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w8/diffusion_arch.png)